In [73]:
import pandas as pd, numpy as np

In [74]:
data1 = pd.read_excel("/content/Tools_1.xlsx", engine='openpyxl') # Changed to read_excel and specified the engine
data2 = pd.read_excel("/content/Tools_2.xlsx", engine='openpyxl')

In [75]:
data1=pd.DataFrame(data1)

# **Basic Preprocessing**

In [76]:
data1.head(3)

,Title,Title_URL,Image,onsale,add_to_wishlist_URL,add_to_wishlist,View,Category,Price,Price1,Type_URL,Type
0,8 Pcs Hegar Uterine Sounds Dilator Set 3mm-18m...,https://surgicalmart.com/shop/surgical-instrum...,https://surgicalmart.com/wp-content/uploads/20...,- 45%,https://surgicalmart.com/product-category/surg...,Add to Wishlist,QUICK VIEW,Gynecology,$39.99,$21.99,https://surgicalmart.com/product-category/surg...,ADD TO CART
1,Metzenbaum Scissors 5.75″ Curved Surgical Vete...,https://surgicalmart.com/shop/surgical-instrum...,https://surgicalmart.com/wp-content/uploads/20...,- 45%,https://surgicalmart.com/product-category/surg...,Add to Wishlist,QUICK VIEW,Surgical Instruments,$10.41,$5.73,https://surgicalmart.com/product-category/surg...,ADD TO CART
2,Adson Tissue Forceps 1×2 Rat Tooth 4.75″ Fine ...,https://surgicalmart.com/shop/surgical-instrum...,https://surgicalmart.com/wp-content/uploads/20...,- 45%,https://surgicalmart.com/product-category/surg...,Add to Wishlist,QUICK VIEW,Dissecting Forceps,$16.35,$8.99,https://surgicalmart.com/product-category/surg...,ADD TO CART


In [88]:
# Check for missing values in the entire DataFrame
data1.isnull().sum()

,0
Title,0
Title_URL,0
Image,0
onsale,1
add_to_wishlist_URL,0
add_to_wishlist,0
View,0
Category,0
Price,0
discounted_price,0


In [78]:
# Identify duplicate ows based on all columns
data1[data1.duplicated()]

,Title,Title_URL,Image,onsale,add_to_wishlist_URL,add_to_wishlist,View,Category,Price,Price1,Type_URL,Type


In [87]:
# Display data types for each column
data1.dtypes

,0
Title,object
Title_URL,object
Image,object
onsale,float64
add_to_wishlist_URL,object
add_to_wishlist,object
View,object
Category,category
Price,float64
discounted_price,float64


In [80]:
data1.shape

(2087, 12)

# **Column Cleaning**

In [81]:
!pip install validators


In [ ]:
import pandas as pd
import validators



# 1. Title: Standardize naming conventions
# data1['Title'] = data1['Title'].str.strip().str.title()
data1 = data1.rename(columns={'Price1': 'discounted_price'})
# 2. Title_URL: Validate URLs or extract product IDs
data1['IsValidURL'] = data1['Title_URL'].apply(lambda x: True if validators.url(x) else False)
data1['Product_ID_from_Title'] = data1['Title'].str.extract(r'(SM\d+)')  # Updated pattern
# 3. Image: Check for missing image URLs and mark them
data1['Image'] = data1['Image'].fillna('Missing_URL')

# 4. onsale: Convert from "-45%" format to numeric
# 4. onsale: Convert from "-45%" format to numeric
data1['onsale'] = data1['onsale'].astype(str).str.replace('%', '').str.replace(' ', '').astype(float) # Added .str.replace(' ', '') to remove spaces and convert to string using astype(str)
# 5. Price/discounted_price: Remove "$" symbols, convert to numeric, and calculate discount percentage
# Removing '$' and ',' before converting to float
# Ensure 'Price' and 'discounted_price' columns are of string type before applying .str
data1['Price'] = data1['Price'].astype(str).str.replace('$', '').str.replace(',', '').astype(float)
data1['discounted_price'] = data1['discounted_price'].astype(str).str.replace('$', '').str.replace(',', '').astype(float)
data1['Discount_Percentage'] = ((data1['Price'] - data1['discounted_price']) / data1['Price']) * 100

# 6. Category: Standardize category names
data1['Category'] = data1['Category'].str.title()
data1['Category'] = data1['Category'].astype('category')
# Display the cleaned DataFrame
print(data1.head())

# **Feature Engineering**

In [83]:
import pandas as pd
import re

# 5. Feature Engineering

# Extract product dimensions/sizes from titles
data1['Dimensions'] = data1['Title'].str.extract(r'(\d+(?:\.\d+)?(?:″|\"|cm|mm))')

# Create flags for products with/without images
data1['Has_Image'] = data1['Image'].apply(lambda x: 0 if x == 'Missing_URL' else 1)

In [84]:
import pandas as pd
import validators
import requests

# 6. Data Validation

# Verify that all URLs are valid and accessible
def is_valid_url(url):
    try:
        response = requests.head(url)
        return response.status_code == 200  # Check for successful response
    except requests.exceptions.RequestException:
        return False

# Filter out rows with invalid URLs (optional)
# data1 = data1[data1['Title_URL'].apply(is_valid_url)]

# Check that sale prices are indeed lower than regular prices
invalid_sale_prices = data1[data1['discounted_price'] >= data1['Price']]
if not invalid_sale_prices.empty:
    print("Warning: Sale prices are not lower than regular prices for some products:")
    print(invalid_sale_prices[['Title', 'Price', 'discounted_price']])

# Ensure all products have at least a title and price
missing_title_price = data1[(data1['Title'].isnull()) | (data1['Price'].isnull())]
if not missing_title_price.empty:
    print("Warning: Some products are missing title or price information:")
    print(missing_title_price[['Title', 'Price']])



                                                  Title  Price
106   4 Pcs Ear Wax Removing Syringe Kidney Bowl & O...    NaN
609   Mayo Hegar Needle Holders 5″, 5.5″, 6″, 7″ 8″,...    NaN
913   Hemostat Forceps Straight & Curved with Grip H...    NaN
1023  T/C Castroviejo Needle Holder Curved Serrated ...    NaN
1084  XTRM CRAFT Tissue Forceps | Lung Holding Intes...    NaN
1348  Mayo-Noble Dissecting Scissors 6.5″ Straight a...    NaN
1351  T/C Mayo Dissecting Scissors 5.5″, 6.75″, 8″, ...    NaN
1352  T/C Mayo Dissecting Scissors 5.5″, 6.75″, 8″, ...    NaN


In [85]:
import pandas as pd
import validators
import requests


# Handling Missing Values

# 1. Removal (if applicable):
# data1 = data1.dropna()  # Remove rows with any missing values
# data1 = data1.drop(columns=['column_with_many_missing'])  # Remove specific columns

# 2. Imputation:
# Numerical features:
data1['Price'] = data1['Price'].fillna(data1['Price'].median()) # Fill missing 'Price' with median
data1['discounted_price'] = data1['discounted_price'].fillna(data1['discounted_price'].median())  # Fill missing 'discounted_price' with median

# Categorical features:
data1['Category'] = data1['Category'].fillna(data1['Category'].mode()[0])  # Fill missing 'Category' with mode

In [86]:
data1.head(3)

,Title,Title_URL,Image,onsale,add_to_wishlist_URL,add_to_wishlist,View,Category,Price,discounted_price,Type_URL,Type,IsValidURL,Product_ID_from_Title,Discount_Percentage,Dimensions,Has_Image
0,8 Pcs Hegar Uterine Sounds Dilator Set 3mm-18m...,https://surgicalmart.com/shop/surgical-instrum...,https://surgicalmart.com/wp-content/uploads/20...,-45.0,https://surgicalmart.com/product-category/surg...,Add to Wishlist,QUICK VIEW,Gynecology,39.99,21.99,https://surgicalmart.com/product-category/surg...,ADD TO CART,True,SM1672,45.011253,3mm,1
1,Metzenbaum Scissors 5.75″ Curved Surgical Vete...,https://surgicalmart.com/shop/surgical-instrum...,https://surgicalmart.com/wp-content/uploads/20...,-45.0,https://surgicalmart.com/product-category/surg...,Add to Wishlist,QUICK VIEW,Surgical Instruments,10.41,5.73,https://surgicalmart.com/product-category/surg...,ADD TO CART,True,SM2056,44.956772,5.75″,1
2,Adson Tissue Forceps 1×2 Rat Tooth 4.75″ Fine ...,https://surgicalmart.com/shop/surgical-instrum...,https://surgicalmart.com/wp-content/uploads/20...,-45.0,https://surgicalmart.com/product-category/surg...,Add to Wishlist,QUICK VIEW,Dissecting Forceps,16.35,8.99,https://surgicalmart.com/product-category/surg...,ADD TO CART,True,SM1621,45.015291,4.75″,1


# **Final**

In [89]:
data1.to_csv('cleaned_surgical_tools.csv', index=False)
